In [0]:
from pyspark.sql.functions import (
    col, to_date, to_timestamp, trim, upper,
    current_timestamp
)

# ADLS Gen2 configuration

spark.conf.set(
    "fs.azure.account.key.healthcarestoragerk.dfs.core.windows.net",
    "xxxxx"
)

bronze_ref_base = "abfss://bronze@healthcarestoragerk.dfs.core.windows.net/reference"
silver_base     = "abfss://silver@healthcarestoragerk.dfs.core.windows.net"


# 1) Load Bronze reference tables

patients_bronze    = spark.read.format("delta").load(f"{bronze_ref_base}/patients_bronze")
conditions_bronze  = spark.read.format("delta").load(f"{bronze_ref_base}/conditions_bronze")
organizations_bronze = spark.read.format("delta").load(f"{bronze_ref_base}/organizations_bronze")
claims_bronze      = spark.read.format("delta").load(f"{bronze_ref_base}/claims_bronze")
claims_tx_bronze   = spark.read.format("delta").load(f"{bronze_ref_base}/claims_transactions_bronze")

print("Bronze reference tables loaded.")


# 2) dim_patient (Silver)

dim_patient = (
    patients_bronze
    .withColumn("patient_id", col("Id"))
    .withColumn("birthdate", to_date("BIRTHDATE", "M/d/yyyy"))
    .withColumn("deathdate", to_date("DEATHDATE", "M/d/yyyy"))
    .withColumn("gender", upper(trim(col("GENDER"))))
    .withColumn("race", trim(col("RACE")))
    .withColumn("ethnicity", trim(col("ETHNICITY")))
    .withColumn("marital_status", trim(col("MARITAL")))
    .withColumn("city", trim(col("CITY")))
    .withColumn("state", trim(col("STATE")))
    .withColumn("zip", trim(col("ZIP")))
    .withColumn("lat", col("LAT").cast("double"))
    .withColumn("lon", col("LON").cast("double"))
    .withColumn("healthcare_expenses", col("HEALTHCARE_EXPENSES").cast("double"))
    .withColumn("healthcare_coverage", col("HEALTHCARE_COVERAGE").cast("double"))
    .withColumn("_silver_ingestion_ts", current_timestamp())
    .select(
        "patient_id",
        "FIRST",
        "LAST",
        "gender",
        "race",
        "ethnicity",
        "marital_status",
        "birthdate",
        "deathdate",
        "BIRTHPLACE",
        "ADDRESS",
        "city",
        "state",
        "zip",
        "lat",
        "lon",
        "healthcare_expenses",
        "healthcare_coverage",
        "_silver_ingestion_ts"
    )
)

dim_patient_path = f"{silver_base}/dim_patient"
(
    dim_patient.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(dim_patient_path)
)
print(f"dim_patient written to {dim_patient_path} with {dim_patient.count()} rows.")


# 3) dim_organization (Silver)

dim_organization = (
    organizations_bronze
    .withColumn("organization_id", col("Id"))
    .withColumn("name", trim(col("NAME")))
    .withColumn("city", trim(col("CITY")))
    .withColumn("state", trim(col("STATE")))
    .withColumn("zip", trim(col("ZIP")))
    .withColumn("lat", col("LAT").cast("double"))
    .withColumn("lon", col("LON").cast("double"))
    .withColumn("revenue", col("REVENUE").cast("double"))
    .withColumn("utilization", col("UTILIZATION").cast("double"))
    .withColumn("_silver_ingestion_ts", current_timestamp())
    .select(
        "organization_id",
        "name",
        "ADDRESS",
        "city",
        "state",
        "zip",
        "lat",
        "lon",
        "PHONE",
        "revenue",
        "utilization",
        "_silver_ingestion_ts"
    )
)

dim_org_path = f"{silver_base}/dim_organization"
(
    dim_organization.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(dim_org_path)
)
print(f"dim_organization written to {dim_org_path} with {dim_organization.count()} rows.")


# 4) conditions_silver

conditions_silver = (
    conditions_bronze
    .withColumn("condition_start_date", to_date("START", "M/d/yyyy"))
    .withColumn("condition_end_date", to_date("STOP", "M/d/yyyy"))
    .withColumn("patient_id", col("PATIENT"))
    .withColumn("encounter_id", col("ENCOUNTER"))
    .withColumn("condition_code", col("CODE"))
    .withColumn("condition_description", col("DESCRIPTION"))
    .withColumn("_silver_ingestion_ts", current_timestamp())
    .select(
        "patient_id",
        "encounter_id",
        "condition_code",
        "condition_description",
        "condition_start_date",
        "condition_end_date",
        "_silver_ingestion_ts"
    )
)

conditions_path = f"{silver_base}/conditions"
(
    conditions_silver.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(conditions_path)
)
print(f"conditions_silver written to {conditions_path} with {conditions_silver.count()} rows.")


# 5) claims_silver (now includes encounter_id via APPOINTMENTID)

claims_silver = (
    claims_bronze
    .withColumn("claim_id", col("Id"))
    .withColumn("patient_id", col("PATIENTID"))
    .withColumn("provider_id", col("PROVIDERID"))
    .withColumn("primary_insurance_id", col("PRIMARYPATIENTINSURANCEID"))
    .withColumn("secondary_insurance_id", col("SECONDARYPATIENTINSURANCEID"))
    .withColumn("department_id", col("DEPARTMENTID"))
    .withColumn("patient_department_id", col("PATIENTDEPARTMENTID"))
    .withColumn("diagnosis_1", col("DIAGNOSIS1"))
    .withColumn("diagnosis_2", col("DIAGNOSIS2"))
    .withColumn("diagnosis_3", col("DIAGNOSIS3"))
    .withColumn("encounter_id", col("APPOINTMENTID"))  # join key to encounters
    .withColumn("current_illness_date", to_timestamp("CURRENTILLNESSDATE"))
    .withColumn("service_date", to_timestamp("SERVICEDATE"))
    .withColumn("status_primary", col("STATUS1"))
    .withColumn("status_secondary", col("STATUS2"))
    .withColumn("status_patient", col("STATUSP"))
    .withColumn("outstanding_primary", col("OUTSTANDING1").cast("double"))
    .withColumn("outstanding_secondary", col("OUTSTANDING2").cast("double"))
    .withColumn("outstanding_patient", col("OUTSTANDINGP").cast("double"))
    .withColumn("_silver_ingestion_ts", current_timestamp())
    .select(
        "claim_id",
        "encounter_id",
        "patient_id",
        "provider_id",
        "primary_insurance_id",
        "secondary_insurance_id",
        "department_id",
        "patient_department_id",
        "diagnosis_1",
        "diagnosis_2",
        "diagnosis_3",
        "current_illness_date",
        "service_date",
        "status_primary",
        "status_secondary",
        "status_patient",
        "outstanding_primary",
        "outstanding_secondary",
        "outstanding_patient",
        "_silver_ingestion_ts"
    )
)

claims_silver_path = f"{silver_base}/claims"
(
    claims_silver.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(claims_silver_path)
)
print(f"claims_silver written to {claims_silver_path} with {claims_silver.count()} rows.")


# 6) claims_transactions_silver

claims_tx_silver = (
    claims_tx_bronze
    .withColumn("transaction_id", col("ID"))
    .withColumn("claim_id", col("CLAIMID"))
    .withColumn("patient_id", col("PATIENTID"))
    .withColumn("transaction_type", col("TYPE"))
    .withColumn("amount", col("AMOUNT").cast("double"))
    .withColumn("payment_method", col("METHOD"))
    .withColumn("from_date", to_timestamp("FROMDATE"))
    .withColumn("to_date", to_timestamp("TODATE"))
    .withColumn("place_of_service", col("PLACEOFSERVICE"))
    .withColumn("procedure_code", col("PROCEDURECODE"))
    .withColumn("units", col("UNITS").cast("double"))
    .withColumn("payments", col("PAYMENTS").cast("double"))
    .withColumn("adjustments", col("ADJUSTMENTS").cast("double"))
    .withColumn("transfers", col("TRANSFERS").cast("double"))
    .withColumn("outstanding", col("OUTSTANDING").cast("double"))
    .withColumn("_silver_ingestion_ts", current_timestamp())
    .select(
        "transaction_id",
        "claim_id",
        "patient_id",
        "transaction_type",
        "amount",
        "payment_method",
        "from_date",
        "to_date",
        "place_of_service",
        "procedure_code",
        "units",
        "payments",
        "adjustments",
        "transfers",
        "outstanding",
        "_silver_ingestion_ts"
    )
)

claims_tx_silver_path = f"{silver_base}/claims_transactions"
(
    claims_tx_silver.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(claims_tx_silver_path)
)
print(f"claims_transactions_silver written to {claims_tx_silver_path} with {claims_tx_silver.count()} rows.")

print("\nSilver dimensions & reference tables built successfully")


Bronze reference tables loaded.
dim_patient written to abfss://silver@healthcarestoragerk.dfs.core.windows.net/dim_patient with 1163 rows.
dim_organization written to abfss://silver@healthcarestoragerk.dfs.core.windows.net/dim_organization with 1127 rows.
conditions_silver written to abfss://silver@healthcarestoragerk.dfs.core.windows.net/conditions with 38094 rows.
claims_silver written to abfss://silver@healthcarestoragerk.dfs.core.windows.net/claims with 117889 rows.
claims_transactions_silver written to abfss://silver@healthcarestoragerk.dfs.core.windows.net/claims_transactions with 711238 rows.

Silver dimensions & reference tables built successfully


In [0]:
display(spark.read.format("delta").load("abfss://silver@healthcarestoragerk.dfs.core.windows.net/dim_patient"))
display(spark.read.format("delta").load("abfss://silver@healthcarestoragerk.dfs.core.windows.net/dim_organization"))


patient_id,FIRST,LAST,gender,race,ethnicity,marital_status,birthdate,deathdate,BIRTHPLACE,ADDRESS,city,state,zip,lat,lon,healthcare_expenses,healthcare_coverage,_silver_ingestion_ts
b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,Damon455,Langosh790,M,white,nonhispanic,null,2019-02-17,null,Middleborough Massachusetts US,620 Lynch Tunnel Apt 0,Springfield,Massachusetts,1104,42.08038942501558,-72.48043144917739,9039.1645,7964.1255,2025-12-28T04:12:30.979081Z
c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,Thi53,Wunsch504,F,white,nonhispanic,null,2005-07-04,null,Danvers Massachusetts US,972 Tillman Branch Suite 48,Bellingham,Massachusetts,null,42.03521335752818,-71.48251904737748,402723.415,14064.135000000002,2025-12-28T04:12:30.979081Z
339144f8-50e1-633e-a013-f361391c4cff,Chi716,Greenfelder433,M,white,nonhispanic,null,1998-05-11,null,Athens Athens Prefecture GR,1060 Bernhard Crossroad Suite 15,Boston,Massachusetts,2131,42.29255662362827,-71.06116042204106,571935.8725,787.5374999999999,2025-12-28T04:12:30.979081Z
d488232e-bf14-4bed-08c0-a82f34b6a197,Phillis443,Walter473,F,white,nonhispanic,null,2003-01-28,null,Boston Massachusetts US,677 Ritchie Terrace,Hingham,Massachusetts,2043,42.200490824321214,-70.91607557520302,582557.803,104782.20699999998,2025-12-28T04:12:30.979081Z
217f95a3-4e10-bd5d-fb67-0cfb5e8ba075,Jerrold404,Herzog843,M,black,nonhispanic,M,1993-12-23,null,Boston Massachusetts US,276 Bernier Branch,Revere,Massachusetts,null,42.3818748756275,-70.99928578468784,475826.855,18067.094999999998,2025-12-28T04:12:30.979081Z
faac724a-a9e9-be66-fe1e-3044dc0ba8ea,Brandon214,Watsica258,F,white,nonhispanic,null,2020-12-04,null,Northborough Massachusetts US,734 Brakus Pathway Unit 28,New Marlborough,Massachusetts,null,42.147028011364384,-73.21710320353982,31468.039999999997,206.21999999999997,2025-12-28T04:12:30.979081Z
23d16ee3-8cd4-eeb8-e77e-1e5fbf4c4159,Rodrigo242,Arellano2,M,black,hispanic,M,1990-12-15,null,Bogota Bogota CO,538 Crona Underpass,Revere,Massachusetts,2151,42.44326150073713,-71.02232158016432,772362.7250000001,2213.085,2025-12-28T04:12:30.979081Z
aade3c61-92bd-d079-9d28-0b2b7fde0fbb,Sal878,Hoppe518,M,white,nonhispanic,M,1985-06-05,null,Merrimac Massachusetts US,930 Goldner Stravenue Apt 8,Somerville,Massachusetts,2138,42.363459871015344,-71.1022528716212,924957.2675000001,874.3724999999998,2025-12-28T04:12:30.979081Z
0288c42c-43a1-9878-4a9d-6b96caa12c40,Miki234,Kozey370,F,white,nonhispanic,M,1979-12-17,null,Northbridge Massachusetts US,636 Stamm Gateway,Cambridge,Massachusetts,2472,42.337425423321065,-71.1066547010483,1134817.5705,226909.95949999997,2025-12-28T04:12:30.979081Z
dc6c06d0-a7d8-100f-c08b-46c93700c188,Taylor21,Gulgowski816,M,white,nonhispanic,null,2006-07-11,null,East Longmeadow Massachusetts US,1037 Hills Extension,Springfield,Massachusetts,1013,42.13505594447831,-72.48174986407135,390675.5774999999,1826.3324999999998,2025-12-28T04:12:30.979081Z


organization_id,name,ADDRESS,city,state,zip,lat,lon,PHONE,revenue,utilization,_silver_ingestion_ts
ef58ea08-d883-3957-8300-150554edc8fb,HEALTHALLIANCE HOSPITALS INC,60 HOSPITAL ROAD,LEOMINSTER,MA,01453,42.520838,-71.770876,9784662000,0.0,1214.0,2025-12-28T04:12:40.323875Z
69176529-fd1f-3b3f-abce-a0a3626769eb,MOUNT AUBURN HOSPITAL,330 MOUNT AUBURN STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275,6174923500,0.0,2877.0,2025-12-28T04:12:40.323875Z
5e765f2b-e908-3888-9fc7-df2cb87beb58,STURDY MEMORIAL HOSPITAL,211 PARK STREET,ATTLEBORO,MA,02703,41.931653,-71.294503,5082225200,0.0,2365.0,2025-12-28T04:12:40.323875Z
f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,LAWRENCE GENERAL HOSPITAL,ONE GENERAL STREET,LAWRENCE,MA,01842,42.700273,-71.161357,9786834000,0.0,976.0,2025-12-28T04:12:40.323875Z
e002090d-4e92-300e-b41e-7d1f21dee4c6,CAMBRIDGE HEALTH ALLIANCE,1493 CAMBRIDGE STREET,CAMBRIDGE,MA,02138,42.375967,-71.118275,6176652300,0.0,2706.0,2025-12-28T04:12:40.323875Z
ef6ab57c-ed94-3dbe-9861-812d515918b3,CAPE COD HOSPITAL,88 LEWIS BAY ROAD,HYANNIS,MA,02601,41.748854,-70.74053599999998,5087711800,0.0,2071.0,2025-12-28T04:12:40.323875Z
49318f80-bd8b-3fc7-a096-ac43088b0c12,COOLEY DICKINSON HOSPITAL INC THE,30 LOCUST STREET,NORTHAMPTON,MA,01060,42.327044,-72.67463000000002,4135822000,0.0,1533.0,2025-12-28T04:12:40.323875Z
fbf6180e-b800-3ebe-b91d-93d0288c400e,BAYSTATE FRANKLIN MEDICAL CENTER,164 HIGH STREET,GREENFIELD,MA,01301,42.614671,-72.597063,4137730211,0.0,1754.0,2025-12-28T04:12:40.323875Z
8b58cdd1-3d79-3126-8fe0-da2c54d6805c,CARNEY HOSPITAL,2100 DORCHESTER AVENUE,BOSTON,MA,02124,42.33196,-71.020173,6175062000,0.0,576.0,2025-12-28T04:12:40.323875Z
4bdaa4c2-c664-3089-aee2-7137abbad27f,HARRINGTON MEMORIAL HOSPITAL-1,100 SOUTH STREET,SOUTHBRIDGE,MA,01550,42.059669,-72.03404,5087659771,0.0,667.0,2025-12-28T04:12:40.323875Z
